In [ ]:
from langchain_core.retrievers import BaseRetriever, RetrieverLike, RetrieverOutputLike
from langchain_core.language_models import BaseLLM
from langchain_core.embeddings import Embeddings
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List
from ragatouille import RAGPretrainedModel



In [ ]:
indexes = []
GROUP_NUM = 10 # 1, 2, or 3
NUM_CLUSTERS = 2


for i in range(NUM_CLUSTERS):
    path_to_index = ".ragatouille/colbert/indexes/GROUP{}_cluster{}".format(GROUP_NUM, i)
    RAG = RAGPretrainedModel.from_index(path_to_index)
    indexes.append(RAG)

In [ ]:
class CustomRetriever(BaseRetriever):
  

    
    vectorstore : List[RetrieverLike]

    def flatten_extend(self, matrix):
        flat_list = []
        for row in matrix:
            flat_list.extend(row)
        return flat_list

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:

        all_docs = []
        for i in self.vectorstore:
            all_docs.append(i.get_relevant_documents(query, k=2))

        all_docs = self.flatten_extend(all_docs)
        return all_docs

In [ ]:
customRetriever = CustomRetriever(vectorstore=indexes)
